# Data Summary and Goal

## Summary

Tanzania, as a developing country, struggles with providing clean water to its population of over 57,000,000. There are many waterpoints already established in the country, but some are in need of repair while others have failed altogether.

## Goal

Build a classifier to predict the condition of a water well, using information provided in the data. This information includes:
- Date
- Location
- Source
- Funder
- And more!

This data is from the DrivenData.org website. It is part of the "Pump It Up: Data Mining the Water Table" dfetition. DrivenData decided to split the data up into two sets, the "Training Set" and the "Test Set". 

It is implied by the names that we are to use the training set for creating our models, and the test set to test them. For this project, we considered merging the two dataframes in order to have more data to work with, however there are 59,400 entries in the training set and therefore more than enough to make good predicitons. 

If our models are subpar, we may merge the tables to aquire more data points to potentially improve model efficacy.

# Data Cleaning

## Import Libraries and Data

In [ ]:
# Import Pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import simplefilter

# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Load data into  Pandas dataframes
status_groups = pd.read_csv('status_groups.csv')
testset = pd.read_csv('test_set.csv')
df = pd.read_csv('training_set.csv')


# Let's add our target series to the dataframe!
status_groups.drop(['id'], axis=1, inplace=True)
df = pd.concat([df, status_groups], axis=1)

# Analyze shape of dataset
print(f'Shape of dataset: {df.shape}')
# display(df.head())


## Drop unneeded columns and deal with missing values

In [ ]:

# Create column that shows the age of the well at the time of recording


from datetime import datetime
df['date_recorded'] = pd.to_datetime(df['date_recorded'])
recorded_year = [x.year for x in df.date_recorded]
df['well_age'] = recorded_year - df.construction_year
# df.well_age.value_counts()
# df['well_age'][df.well_age<0]
# df.iloc[[10441, 8729, 13366, 23373, 27501, 32619, 33942, 39559]][['construction_year', 'date_recorded', 'status_group']]


# Interestingly enough, there are some negative values for the age of wells
# This indicates that the well was PLANNED on being built at the time of recording, but had not yet been recorded

In [ ]:
# EXPLORATORY! We are analyzing the columns in order to remove redundancies
# **Uncomment if viewing the unique values is desired**

# df.region.unique()
# df.scheme_management

# display(df['extraction_type_class'].unique())
# display(df['extraction_type'].unique())
# display(df['extraction_type_group'].unique())

# display(df['source_class'].unique())
# display(df['source'].unique())
# display(df['source_type'].unique())

# display(df.waterpoint_type.unique())
# display(df.waterpoint_type_group.unique())

# display(df.water_quality.unique())
# display(df.quality_group.unique())

# display(df.management.unique())
# display(df.management_group.unique())

# display(df.payment.unique())
# display(df.payment_type.unique())

# display(df.quantity.unique())
# display(df.quantity_group.unique())

df.isna().sum()[df.isna().sum()>0]

In [ ]:
# Label encode the target variable
status_labels = {'status_group':{'non functional': 0, 'functional': 1, 'functional needs repair': 2}}
df = df.replace(status_labels)
df.status_group.value_counts()

# Organize the funder column a little based on occurence
low_occurence = list(df.funder[df['funder'].map(df['funder'].value_counts()) < 100].values)
df['funder'].replace(low_occurence, 'other', inplace=True)

# Drop redundant and unneeded columns
to_drop = ['scheme_name', 'recorded_by', 'wpt_name', 'extraction_type', 'extraction_type_group',
           'region_code', 'district_code', 'latitude', 'longitude', 'lga', 'ward', 'public_meeting', 'date_recorded', 
           'source', 'source_type', 'waterpoint_type', 'water_quality', 'management_group', 
           'payment', 'quantity_group','subvillage', 'num_private', 'construction_year']

# Deal with missing values
df.drop(to_drop, axis=1, inplace=True)
df.scheme_management.replace({'None':'Ignore', np.nan:'Ignore'}, inplace=True)
df.permit.fillna(False, inplace=True)
df.dropna(axis=0, inplace=True)

# Set 'id' as the index of the dataframe
df.set_index('id', inplace=True)

## Grouping and Labeling Column Values

In [ ]:
# Permit

df.permit.replace({True:1, False:0}, inplace=True)

In [ ]:
# Population
    
def population(obs):
    s=''
    x=obs['population']
    if(0<x<=100):
        s='Less than 100'
    elif(100<x<=200):
        s='Between 100 and 200'
    elif(200<x<=300):
        s='Between 200 and 300'
    elif(300<x<=400):
        s='between 300 and 400'
    elif(400<x<=500):
        s='between 400 and 500'
    elif(500<x):
        s='Over 500'
    elif(x==0):
        s='No population'
    return s
df['population']=df.apply(population,axis=1)


In [ ]:
# Well_age

# Drop all items that have a value less than 0 (very few)
df.drop(df[df['well_age'] < 0].index, inplace = True)

# Bin
conditions = [df.well_age==0, (df.well_age>0)&(df.well_age<=4), (df.well_age>4)&(df.well_age<=12), (df.well_age>12)&(df.well_age<=25), 
              (df.well_age>25)&(df.well_age<=48), df.well_age>48]
choices = ['new', '0-4 years', '4-12 years', '12-25 years', '25-48 years', 'more than 48 years']
df['well_age'] = np.select(conditions, choices)

In [ ]:
# Amount_tsh
# Bin
conditions = [df.amount_tsh==0,(df.amount_tsh>0)&(df.amount_tsh<=10),(df.amount_tsh>10)&(df.amount_tsh<=100), (df.amount_tsh>100)&(df.amount_tsh<=1000),
             (df.amount_tsh>1000)&(df.amount_tsh<=2000), (df.amount_tsh>2000)&(df.amount_tsh<=10000), (df.amount_tsh>10000)&(df.amount_tsh<=100000),
             df.amount_tsh>100000]
choices = ['zero', '1 to 10', '11 to 100', '101 to 1k', '1k to 2k', '2k to 10k', '10k to 100k', 'greater than 100k']
df['amount_tsh'] = np.select(conditions, choices)

In [ ]:
df.gps_height = pd.qcut(df.gps_height, 8, duplicates='drop', 
        labels=['-90m - sea level', 'sea level to 46m', '46m to 393m', '393m to 1017m', '1017m to 1316m', '1316m to 1586.75m', '1586.75m to 2770m'])

# Visualizations

In [ ]:
sns.histplot(target, stat='density')

In [ ]:
df.population.value_counts().plot(kind='bar')

In [ ]:
df.well_age.value_counts().plot(kind='bar')

# Model Building

In [ ]:
# Identify features and target
features = df.drop('status_group', axis=1)
target = df.status_group

# Dummy the features
pd.get_dummies(cats, drop_first=True)

# Split the data
from sklearn.model_selection import train_test_split
X_train, y_train, X_test, y_test = train_test_split(features, target, random_state=33)